In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TrainManager Class
def load_checkpoint(model, checkpoint_path):
    model_ckp = torch.load(checkpoint_path)
    model.load_state_dict(model_ckp['model_state_dict'])
    return model


class TrainManager(object):
		def __init__(self, student, teacher=None, train_loader=None, test_loader=None, train_config={}):
				self.student = student
				self.teacher = teacher
				self.have_teacher = bool(self.teacher)
				self.device = train_config['device']
				self.name = train_config['name']
				self.optimizer = optim.SGD(self.student.parameters(),
											lr=train_config['learning_rate'],
											momentum=train_config['momentum'],
											weight_decay=train_config['weight_decay'])
				if self.have_teacher:
						self.teacher.eval()
						self.teacher.train(mode=False)
					
				self.train_loader = train_loader
				self.test_loader = test_loader
				self.config = train_config
		
		def train(self):
				epochs = self.config['epochs']
				trial_id = self.config['trial_id']
				
				max_val_acc = 0
				iteration = 0
				best_acc = 0
				criterion = nn.CrossEntropyLoss()
				
				training_time = time.time()
				for epoch in range(1, epochs+1):
						start_time = time.time()
						self.student.train()
						self.adjust_learning_rate(self.optimizer, epoch)
						loss = 0
						for batch_idx, (data, target) in enumerate(self.train_loader):
								iteration += 1
								data = data.to(self.device)
								target = target.to(self.device)
								self.optimizer.zero_grad()
								output = self.student(data)
								# Standard Learning Loss ( Classification Loss)
								loss_SL = criterion(output, target) 
								loss = loss_SL
								
								if self.have_teacher:
										teacher_outputs = self.teacher(data)
										# Knowledge Distillation Loss
										loss_KD = nn.KLDivLoss()(F.log_softmax(output / T, dim=1),
																			F.softmax(teacher_outputs / T, dim=1))
										loss = (1 - lambda_) * loss_SL + lambda_ * T * T * loss_KD
								loss.backward()
								self.optimizer.step()
					
						print("epoch {0}/{1} --- {2:0.3f} seconds --- ".format(epoch, epochs, time.time() - start_time))
						val_acc = self.validate(step=epoch)
						if val_acc > best_acc:
								best_acc = val_acc
								best_epoch = epoch
								self.save(epoch, name='{}_{}_best.pth.tar'.format(self.name, trial_id))

				teacher_model = create_cnn_model(self.name, 'cifar10', use_cuda=True)
				teacher_model = load_checkpoint(teacher_model, os.path.join('./','{}_{}_best.pth.tar'.format(self.name, trial_id)))
				self.save(best_epoch, name='{}_{}_{}_best.pth.tar'.format(best_acc,self.name, trial_id))
				print("Total elapsed time : {0:0.3f} seconds --- ".format(time.time() - training_time))
				print("Best accuracy was : {}".format(best_acc))
				return best_acc
		
		def validate(self, step=0):
				self.student.eval()
				with torch.no_grad():
						correct = 0
						total = 0
						acc = 0
						for images, labels in self.test_loader:
								images = images.to(self.device)
								labels = labels.to(self.device)
								outputs = self.student(images)
								_, predicted = torch.max(outputs.data, 1)
								total += labels.size(0)
								correct += (predicted == labels).sum().item()
						acc = 100 * correct / total
						
						print('{{"metric": "{}_val_accuracy", "value": {}}}'.format(self.name, acc))
						return acc
		
		def save(self, epoch, name=None):
				trial_id = self.config['trial_id']
				if name is None:
						torch.save({
							'epoch': epoch,
							'model_state_dict': self.student.state_dict(),
							'optimizer_state_dict': self.optimizer.state_dict(),
						}, '{}_{}_epoch{}.pth.tar'.format(self.name, trial_id, epoch))
				else:
						torch.save({
							'model_state_dict': self.student.state_dict(),
							'optimizer_state_dict': self.optimizer.state_dict(),
							'epoch': epoch,
						}, name)
		
		def adjust_learning_rate(self, optimizer, epoch):
				epochs = self.config['epochs']
				models_are_plane = self.config['is_plane']
				
				# depending on dataset
				if models_are_plane:
						lr = 0.01
				else:
						if epoch < int(epoch/2.0):
								lr = 0.1
						elif epoch < int(epochs*3/4.0):
								lr = 0.1 * 0.1
						else:
								lr = 0.1 * 0.01
				
				# update optimizer's learning rate
				for param_group in optimizer.param_groups:
						param_group['lr'] = lr

In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/DLF/teacher')
import copy
import math
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as torch_models
import torch.nn.functional as F
import torch.optim as optim
import time
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/DLF/TaKD')

import data_loader
import model_factory
import plain_cnn_cifar
import resnet_cifar

from data_loader import get_cifar
from model_factory import create_cnn_model, is_resnet

from resnet_cifar import *
from plain_cnn_cifar import *

In [5]:
myname = "CH"
batch_size = 128

dataset = 'cifar10'
num_classes = 10 # 10 or 100
train_loader, test_loader = get_cifar(num_classes, dataset_dir='/content/drive/My Drive/Colab Notebooks/DLF/cifar/crop', batch_size=batch_size, crop = True)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
teacher = "resnet26"
epochs = 80
trial_num = 1
chseed = 20 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': teacher
	}

teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)

print("---------- Training Teacher -------")
teacher_name = '{}_{}_best.pth.tar'.format(teacher, trial_id)
print("teacher name is " + teacher_name)
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
teacher_trainer.train()

---------- Training Teacher -------
teacher name is resnet26_CH_20_128 # 1_best.pth.tar
epoch 1/80 --- 18.487 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 42.14}
epoch 2/80 --- 18.257 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 53.59}
epoch 3/80 --- 17.821 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 62.88}
epoch 4/80 --- 17.786 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 67.4}
epoch 5/80 --- 18.036 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 64.31}
epoch 6/80 --- 18.091 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 71.43}
epoch 7/80 --- 18.017 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 73.89}
epoch 8/80 --- 18.034 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 74.73}
epoch 9/80 --- 17.930 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 75.9}
epoch 10/80 --- 17.932 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 77.56}
epoch 11/80 --- 18.017 seconds --- 
{"metri

89.28

In [0]:
teacher = "resnet26"
epochs = 80
trial_num = 2
chseed = 20 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': teacher
	}

teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)

print("---------- Training Teacher -------")
teacher_name = '{}_{}_best.pth.tar'.format(teacher, trial_id)
print("teacher name is " + teacher_name)
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
teacher_trainer.train()

---------- Training Teacher -------
teacher name is resnet26_CH_20_128 # 2_best.pth.tar
epoch 1/80 --- 18.047 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 43.12}
epoch 2/80 --- 17.923 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 55.76}
epoch 3/80 --- 18.109 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 62.88}
epoch 4/80 --- 18.007 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 66.25}
epoch 5/80 --- 17.932 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 69.82}
epoch 6/80 --- 17.898 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 70.69}
epoch 7/80 --- 17.942 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 70.67}
epoch 8/80 --- 17.932 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 74.71}
epoch 9/80 --- 17.922 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 76.89}
epoch 10/80 --- 17.982 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 77.45}
epoch 11/80 --- 17.956 seconds --- 
{"met

89.35

In [0]:
teacher = "resnet26"
epochs = 80
trial_num = 3
chseed = 31 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': teacher
	}

teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)

print("---------- Training Teacher -------")
teacher_name = '{}_{}_best.pth.tar'.format(teacher, trial_id)
print("teacher name is " + teacher_name)
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
teacher_trainer.train()

---------- Training Teacher -------
teacher name is resnet26_CH_31_128 # 3_best.pth.tar
epoch 1/80 --- 17.933 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 45.48}
epoch 2/80 --- 17.867 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 54.14}
epoch 3/80 --- 17.834 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 64.1}
epoch 4/80 --- 17.932 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 65.63}
epoch 5/80 --- 17.917 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 63.77}
epoch 6/80 --- 17.937 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 68.11}
epoch 7/80 --- 17.986 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 73.75}
epoch 8/80 --- 17.930 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 73.95}
epoch 9/80 --- 17.884 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 74.83}
epoch 10/80 --- 17.938 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 76.77}
epoch 11/80 --- 17.904 seconds --- 
{"metr

89.41

In [0]:
teacher = "resnet26"
epochs = 80
trial_num = 4
chseed = 31 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': teacher
	}

teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)

print("---------- Training Teacher -------")
teacher_name = '{}_{}_best.pth.tar'.format(teacher, trial_id)
print("teacher name is " + teacher_name)
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
teacher_trainer.train()

---------- Training Teacher -------
teacher name is resnet26_CH_31_128 # 4_best.pth.tar
epoch 1/80 --- 13.124 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 41.84}
epoch 2/80 --- 13.197 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 54.99}
epoch 3/80 --- 13.029 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 63.88}
epoch 4/80 --- 12.961 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 65.14}
epoch 5/80 --- 13.032 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 66.45}
epoch 6/80 --- 13.047 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 68.07}
epoch 7/80 --- 13.173 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 73.33}
epoch 8/80 --- 13.157 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 74.45}
epoch 9/80 --- 13.123 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 74.65}
epoch 10/80 --- 12.967 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 76.18}
epoch 11/80 --- 13.015 seconds --- 
{"met

88.92

In [0]:
teacher = "resnet26"
epochs = 80
trial_num = 5
chseed = 55 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': teacher
	}

teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)

print("---------- Training Teacher -------")
teacher_name = '{}_{}_best.pth.tar'.format(teacher, trial_id)
print("teacher name is " + teacher_name)
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
teacher_trainer.train()

---------- Training Teacher -------
teacher name is resnet26_CH_55_128 # 5_best.pth.tar
epoch 1/80 --- 37.653 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 44.45}
epoch 2/80 --- 38.341 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 55.44}
epoch 3/80 --- 38.277 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 60.67}
epoch 4/80 --- 38.229 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 64.33}
epoch 5/80 --- 38.332 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 64.96}
epoch 6/80 --- 38.312 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 67.95}
epoch 7/80 --- 38.214 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 71.56}
epoch 8/80 --- 38.295 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 69.87}
epoch 9/80 --- 38.337 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 75.86}
epoch 10/80 --- 38.219 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 76.67}
epoch 11/80 --- 38.240 seconds --- 
{"met

89.25

In [0]:
teacher = "resnet26"
epochs = 80
trial_num = 6
chseed = 55 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': teacher
	}

teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)

print("---------- Training Teacher -------")
teacher_name = '{}_{}_best.pth.tar'.format(teacher, trial_id)
print("teacher name is " + teacher_name)
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
teacher_trainer.train()

In [6]:
teacher = "resnet26"
epochs = 80
trial_num = 7
chseed = 16 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': teacher
	}

teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)

print("---------- Training Teacher -------")
teacher_name = '{}_{}_best.pth.tar'.format(teacher, trial_id)
print("teacher name is " + teacher_name)
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
teacher_trainer.train()

---------- Training Teacher -------
teacher name is resnet26_CH_16_128 # 7_best.pth.tar
epoch 1/80 --- 15.130 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 45.24}
epoch 2/80 --- 14.212 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 54.43}
epoch 3/80 --- 14.378 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 60.88}
epoch 4/80 --- 14.793 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 65.27}
epoch 5/80 --- 14.851 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 64.74}
epoch 6/80 --- 14.638 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 68.33}
epoch 7/80 --- 14.647 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 73.18}
epoch 8/80 --- 15.164 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 77.19}
epoch 9/80 --- 14.796 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 77.4}
epoch 10/80 --- 14.712 seconds --- 
{"metric": "resnet26_val_accuracy", "value": 77.14}
epoch 11/80 --- 14.708 seconds --- 
{"metr

89.06

In [0]:
teacher = "resnet26"
epochs = 80
trial_num = 8
chseed = 101 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': teacher
	}

teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)

print("---------- Training Teacher -------")
teacher_name = '{}_{}_best.pth.tar'.format(teacher, trial_id)
print("teacher name is " + teacher_name)
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
teacher_trainer.train()

In [0]:
teacher = "resnet26"
epochs = 80
trial_num = 9
chseed = 79 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': teacher
	}

teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)

print("---------- Training Teacher -------")
teacher_name = '{}_{}_best.pth.tar'.format(teacher, trial_id)
print("teacher name is " + teacher_name)
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
teacher_trainer.train()

In [0]:
teacher = "resnet26"
epochs = 80
trial_num = 10
chseed = 43 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': teacher
	}

teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)

print("---------- Training Teacher -------")
teacher_name = '{}_{}_best.pth.tar'.format(teacher, trial_id)
print("teacher name is " + teacher_name)
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
teacher_trainer.train()

In [0]:
teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)

print("---------- Training Teacher -------")
teacher_name = '{}_{}_best.pth.tar'.format(teacher, trial_id)
print("teacher name is " + teacher_name)
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
teacher_trainer.train()

---------- Training Teacher -------
teacher name is resnet20_CH_55_256 # 3_best.pth.tar
epoch 1/80 --- 27.884 seconds --- 
{"metric": "resnet20_val_accuracy", "value": 36.52}
epoch 2/80 --- 27.739 seconds --- 
{"metric": "resnet20_val_accuracy", "value": 45.49}
epoch 3/80 --- 27.749 seconds --- 
{"metric": "resnet20_val_accuracy", "value": 52.49}
epoch 4/80 --- 27.753 seconds --- 
{"metric": "resnet20_val_accuracy", "value": 54.35}
epoch 5/80 --- 27.768 seconds --- 
{"metric": "resnet20_val_accuracy", "value": 61.49}
epoch 6/80 --- 27.725 seconds --- 
{"metric": "resnet20_val_accuracy", "value": 63.24}
epoch 7/80 --- 27.768 seconds --- 
{"metric": "resnet20_val_accuracy", "value": 67.22}
epoch 8/80 --- 27.737 seconds --- 
{"metric": "resnet20_val_accuracy", "value": 68.57}
epoch 9/80 --- 27.782 seconds --- 
{"metric": "resnet20_val_accuracy", "value": 68.38}
epoch 10/80 --- 27.707 seconds --- 
{"metric": "resnet20_val_accuracy", "value": 71.57}
epoch 11/80 --- 27.723 seconds --- 
{"met

87.24

In [0]:
#끊겼을떄 다시시작
args.teacher = "resnet8"

teacher_model = create_cnn_model(args.teacher, dataset, use_cuda=args.cuda)
teacher_name = '{}_{}_best.pth.tar'.format(args.teacher, trial_id)
teacher_model = load_checkpoint(teacher_model, os.path.join('./', teacher_name))

print("---------- Training Teacher -------")
train_loader, test_loader = get_cifar(num_classes, dataset_dir='/content/drive/My Drive/Colab Notebooks/DLF/cifar')
teacher_train_config = copy.deepcopy(train_config)
teacher_train_config['epochs'] = 1 #additional epochs
teacher_train_config['name'] = args.teacher
teacher_trainer = TrainManager(teacher_model, teacher=None, train_loader=train_loader, test_loader=test_loader, train_config=teacher_train_config)
teacher_trainer.train()

#teacher_model = load_checkpoint(teacher_model, os.path.join('./', teacher_name))

args.teacher = ''

---------- Training Teacher -------
Files already downloaded and verified
Files already downloaded and verified
epoch 0/1
{"metric": "resnet8_val_accuracy", "value": 75.96}
